# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
orders = pd.read_csv('Orders.csv')


In [3]:
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [4]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397924 entries, 0 to 397923
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    397924 non-null  int64  
 1   InvoiceNo     397924 non-null  int64  
 2   StockCode     397924 non-null  object 
 3   year          397924 non-null  int64  
 4   month         397924 non-null  int64  
 5   day           397924 non-null  int64  
 6   hour          397924 non-null  int64  
 7   Description   397924 non-null  object 
 8   Quantity      397924 non-null  int64  
 9   InvoiceDate   397924 non-null  object 
 10  UnitPrice     397924 non-null  float64
 11  CustomerID    397924 non-null  int64  
 12  Country       397924 non-null  object 
 13  amount_spent  397924 non-null  float64
dtypes: float64(2), int64(8), object(4)
memory usage: 42.5+ MB


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [5]:
#cuartiles y percentiles IQR

In [6]:
# Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

customer = orders.groupby('CustomerID').sum() #hacemos la suma, xq sino no nos sale la tabla, solo la descripcion 
customer

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,Country,amount_spent
CustomerID,,,,,,,,,,,,,
12346,61619,541431,23166,2011,1,2,10,medium ceramic top storage jar,74215,2011-01-18 10:01:00,1.04,United Kingdom,77183.60
12347,42441700,101296926,8511622375714772249222771227722277322774227752...,365971,1383,441,2219,black candelabra t-light holderairline bag vin...,2458,2010-12-07 14:57:002010-12-07 14:57:002010-12-...,481.21,IcelandIcelandIcelandIcelandIcelandIcelandIcel...,4310.00
12348,2807120,16869685,8499222951849918499121213212132261621981219822...,62324,257,111,472,72 sweetheart fairy cake cases60 cake cases do...,2341,2010-12-16 19:09:002010-12-16 19:09:002010-12-...,178.71,FinlandFinlandFinlandFinlandFinlandFinlandFinl...,1797.24
12349,35444274,42165457,2311223460215642141121563221312219548194849782...,146803,803,73,657,parisienne curio cabinetsweetheart wall tidy p...,631,2011-11-21 09:51:002011-11-21 09:51:002011-11-...,605.10,ItalyItalyItalyItalyItalyItalyItalyItalyItalyI...,1757.55
12350,1365627,9231629,219082241279066K79191C2234884086C2255122557218...,34187,34,51,272,chocolate this way metal signmetal sign neighb...,197,2011-02-02 16:01:002011-02-02 16:01:002011-02-...,65.30,NorwayNorwayNorwayNorwayNorwayNorwayNorwayNorw...,334.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280,1110495,5457120,8248422180224672272522727224952235822499226112...,20110,30,10,90,wood black board ant white finishretrospot lam...,45,2011-03-07 09:52:002011-03-07 09:52:002011-03-...,47.65,United KingdomUnited KingdomUnited KingdomUnit...,180.60
18281,1560699,3895248,22037227162202823007230082320922467,14077,42,49,70,robot birthday cardcard circus paradepenny far...,54,2011-06-12 10:53:002011-06-12 10:53:002011-06-...,39.36,United KingdomUnited KingdomUnited KingdomUnit...,80.82
18282,4642134,6838540,2127023187232952208921108211092242422423226992...,24132,116,60,146,antique cream cutlery cupboardfrench style sto...,103,2011-08-05 13:35:002011-08-05 13:35:002011-08-...,62.39,United KingdomUnited KingdomUnited KingdomUnit...,178.05


In [7]:
len(orders['CustomerID'].unique()) #quiero ver la cantidad numerica de valores unicos

4339

In [8]:
customer['amount_spent'].sort_values(ascending=False) # volviendo a la tabla filtrada de arriba por cliente, comparandola con la de cantidad gastada, la ordeno de manera descendente. 
#De esta manera podre identificar los clientes que mas han gastado


CustomerID
14646    280206.02
18102    259657.30
17450    194550.79
16446    168472.50
14911    143825.06
           ...    
17956        12.75
16454         6.90
14792         6.20
16738         3.75
13256         0.00
Name: amount_spent, Length: 4339, dtype: float64

In [9]:
# Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?
'''
Nos fijamos en los clientes con cantidad gastada desde percentil 75% en adelante, y valoramos sobre esos datos 
'''

# Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

'''
Vip/Promotores = 95%-100%
Preferred/Pasivos = 75%-95%
Restantes/Detractores = 0%-75%
'''

'\nVip/Promotores = 95%-100%\nPreferred/Pasivos = 75%-95%\nRestantes/Detractores = 0%-74%\n'

In [10]:
customer.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,4339.0,2.553768e+07,7.157521e+07,252.0,3569287.500,11090934.00,2.814387e+07,2.405301e+09
InvoiceNo,4339.0,5.141346e+07,1.287813e+08,536393.0,9534986.000,23128573.00,5.630215e+07,4.418581e+09
year,4339.0,1.844201e+05,4.600899e+05,2010.0,34187.000,82451.00,2.011000e+05,1.578003e+07
month,4339.0,6.981357e+02,1.878042e+03,1.0,110.000,308.00,7.695000e+02,6.260700e+04
day,4339.0,3.314861e+02,7.893480e+02,1.0,50.000,136.00,3.550000e+02,2.670400e+04
hour,4339.0,1.167291e+03,2.991085e+03,8.0,204.000,517.00,1.256000e+03,1.038310e+05
Quantity,4339.0,1.194214e+03,5.055603e+03,1.0,160.000,379.00,9.930000e+02,1.974910e+05
UnitPrice,4339.0,2.857803e+02,8.805457e+02,0.0,51.685,127.15,2.944250e+02,3.333245e+04
amount_spent,4339.0,2.053793e+03,8.988248e+03,0.0,307.245,674.45,1.661640e+03,2.802060e+05


In [11]:
customer.describe().T.loc['amount_spent']['75%'] #No me interesan los que se encuentre por debajo del percentil 75

1661.64

In [12]:
clients = customer[customer['amount_spent'] > customer.describe().T.loc['amount_spent']['75%']] #tenemos 1085 clientes dentro del percentil 75
clients

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,Country,amount_spent
CustomerID,,,,,,,,,,,,,
12346,61619,541431,23166,2011,1,2,10,medium ceramic top storage jar,74215,2011-01-18 10:01:00,1.04,United Kingdom,77183.60
12347,42441700,101296926,8511622375714772249222771227722277322774227752...,365971,1383,441,2219,black candelabra t-light holderairline bag vin...,2458,2010-12-07 14:57:002010-12-07 14:57:002010-12-...,481.21,IcelandIcelandIcelandIcelandIcelandIcelandIcel...,4310.00
12348,2807120,16869685,8499222951849918499121213212132261621981219822...,62324,257,111,472,72 sweetheart fairy cake cases60 cake cases do...,2341,2010-12-16 19:09:002010-12-16 19:09:002010-12-...,178.71,FinlandFinlandFinlandFinlandFinlandFinlandFinl...,1797.24
12349,35444274,42165457,2311223460215642141121563221312219548194849782...,146803,803,73,657,parisienne curio cabinetsweetheart wall tidy p...,631,2011-11-21 09:51:002011-11-21 09:51:002011-11-...,605.10,ItalyItalyItalyItalyItalyItalyItalyItalyItalyI...,1757.55
12352,22083029,47523155,2138022064212322264622779224232265422120217552...,170935,552,243,1193,wooden happy birthday garlandpink doughnut tri...,536,2011-02-16 12:33:002011-02-16 12:33:002011-02-...,1354.11,NorwayNorwayNorwayNorwayNorwayNorwayNorwayNorw...,2506.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18259,13890450,23741242,2269422470228672283422112221142283722834228672...,84455,427,133,553,wicker star heart of wicker largehand warmer b...,714,2010-12-08 13:38:002010-12-08 13:38:002010-12-...,136.90,United KingdomUnited KingdomUnited KingdomUnit...,2338.60
18260,18050916,73457520,7932122452224532245421258212572208721843215272...,269463,577,436,1616,chilli lightsmeasuring tape babushka pinkmeasu...,1478,2010-12-16 18:23:002010-12-16 18:23:002010-12-...,469.94,United KingdomUnited KingdomUnited KingdomUnit...,2643.20
18272,49405922,93342091,2075421563714592255722979229802075220751229892...,333826,1221,509,2107,retrospot red washing up glovesred heart shape...,2050,2011-04-07 09:35:002011-04-07 09:35:002011-04-...,380.91,United KingdomUnited KingdomUnited KingdomUnit...,3078.58


In [13]:
clients_len = len(clients['amount_spent']) #confirmamos lo de arriba
clients_len

1085

In [14]:
vip = int(clients_len * 0.05) #aqui separamos el percentil 95% del 75%
preferred = clients_len - vip #creamos otra variable para la resta de los distintos clientes

vip, preferred #efectivamente la suma de ambos nos da el resultado inicial 1085

(54, 1031)

In [15]:
clients['amount_spent'].sort_values() # tabla filtrada por clientes del top 75%, comparandola con la de cantidad gastada

CustomerID
13012      1661.84
12530      1662.28
12912      1662.30
16115      1667.97
17656      1674.69
           ...    
14911    143825.06
16446    168472.50
17450    194550.79
18102    259657.30
14646    280206.02
Name: amount_spent, Length: 1085, dtype: float64

In [16]:
preferred

1031

In [17]:
clients['amount_spent'].sort_values().index#[:preferred]

Index([13012, 12530, 12912, 16115, 17656, 13149, 13647, 16638, 13577, 13162,
       ...
       12346, 16029, 17511, 14156, 12415, 14911, 16446, 17450, 18102, 14646],
      dtype='int64', name='CustomerID', length=1085)

In [18]:
num_vips = round(len(orders.CustomerID.unique()) * 0.05)
num_pref = round(len(orders.CustomerID.unique()) * 0.2)
num_vips, num_pref

(217, 868)

In [19]:
vip_index=list(orders.groupby("CustomerID").sum()["amount_spent"].sort_values(ascending=False).iloc[:num_vips].index)
pref_index = list(orders.groupby("CustomerID").sum()["amount_spent"].sort_values(ascending=False).iloc[217:1085].index)

In [20]:
vips = orders.groupby("CustomerID").sum()["amount_spent"].sort_values(ascending=False).iloc[:num_vips]
vips


CustomerID
14646    280206.02
18102    259657.30
17450    194550.79
16446    168472.50
14911    143825.06
           ...    
14505      5905.64
14415      5900.28
14944      5900.25
13209      5891.61
13534      5870.08
Name: amount_spent, Length: 217, dtype: float64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [25]:
country = orders[['CustomerID', 'Country']]

country

,CustomerID,Country
0,17850,United Kingdom
1,17850,United Kingdom
2,17850,United Kingdom
3,17850,United Kingdom
4,17850,United Kingdom
...,...,...
397919,12680,France
397920,12680,France
397921,12680,France
397922,12680,France


In [22]:
'''
vips = (orders.CustomerID.unique()) * 0.05
vips


country_vip=orders[[vips, 'Country']]
'''
'''
country_vip=orders.groupby('Country')
country_vip
'''
# tabla segun pais (.groupby['Country']),  relacionado con lso vips


In [43]:
country2=orders.iloc[vip_index][['Country']]
country2

,Country
14646,United Kingdom
18102,United Kingdom
17450,United Kingdom
16446,United Kingdom
14911,United Kingdom
...,...
14505,United Kingdom
14415,United Kingdom
14944,United Kingdom
13209,France


In [60]:
'''
country3=country2.groupby('Country').size().sort_values(ascending=False)
Esto es lo mismo
'''

country3=orders.iloc[vip_index][['Country']].groupby('Country').size().sort_values(ascending=False)
country3



Country
United Kingdom    191
France              6
Germany             6
Norway              6
EIRE                3
Portugal            2
Spain               2
Japan               1
dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [68]:
countrycombo=orders.iloc[vip_index+pref_index][['Country']].groupby('Country').size().sort_values(ascending=False)
countrycombo

Country
United Kingdom     971
Norway              24
Germany             22
France              19
Spain               15
EIRE                12
Denmark              8
Japan                6
Portugal             6
Channel Islands      2
dtype: int64

In [69]:
winner2=countrycombo.index[0]
winner2

'United Kingdom'